In [4]:
# Import Librarys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dataprep as dp
import pyarrow.parquet as pq
from dataprep.eda import create_report
from surprise.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
import gradio as gr
import joblib

In [5]:
data_final = pd.read_parquet("./movies_completo_ML.parquet")
# Cargar el modelo de Machine Learning guardado con Joblib
modelo = joblib.load('./modelo.joblib')

In [3]:
data_final.head(5)

,id,userId,score,title
0,ns2186,1,4.5,horse tribe
1,ns2186,231,4.0,horse tribe
2,ns2186,843,5.0,horse tribe
3,ns2186,978,4.5,horse tribe
4,ns2186,1355,4.0,horse tribe


In [6]:
reader = Reader(rating_scale=(1, 5))


In [7]:
dataset = Dataset.load_from_df(data_final[['userId', 'id', 'score']], reader)


In [7]:
dataset

In [8]:
# Entrenamos el modelo

#trainset, testset = train_test_split(dataset, test_size=0.25)
data_train = dataset.build_full_trainset()

In [9]:
algo = SVD()
algo.fit(data_train)

In [11]:
joblib.dump(algo, 'modelo.sav')

['modelo.sav']

In [10]:
data_test = data_train.build_testset()

#data_test_surp = data_test_surp.construct_testset()
predictions = algo.test(data_test)



MAE:  0.4717


0.4717279452240571

In [11]:
accuracy.mae(predictions)
accuracy.rmse(predictions)

MAE:  0.4717
RMSE: 0.6254


0.6253888782841461

In [12]:
user_id = '543'
movie_id = 'as1'
predicted_rating = algo.predict(user_id, movie_id).est

In [13]:
predicted_rating

3.5090921702442386

In [14]:
user_id = '543'
movie_ids = dataset.build_full_trainset().all_items()  # Todas las películas en el conjunto de datos
ratings = [algo.predict(user_id, movie_id).est for movie_id in movie_ids]
sorted_items = [movie_id for _, movie_id in sorted(zip(ratings, movie_ids), reverse=True)]
recommended_items = sorted_items[:10]  # Las 10 películas más recomendadas


In [15]:
user_id = '2523'
movie_id = 'as1'
threshold = 3.5  # Umbral de calificación

predicted_rating = algo.predict(user_id, movie_id).est
if predicted_rating > threshold:
    print(f'{movie_id} es recomendado para el usuario {user_id}.')
else:
    print(f'{movie_id} no es recomendado para el usuario {user_id}.')

as1 es recomendado para el usuario 2523.


In [1]:
def predict_rating(userId, id):
    prediction = modelo.predict(userId, id)

    if prediction.est >= 3.5:
        mensaje = "Es hora de verla!!", prediction.est
    elif prediction.est >= 2.5 and prediction.est < 3.5:
        mensaje = "Puedes verla luego!", prediction.est
    else:
        mensaje = "No es la película que esta buscando!", prediction.est

    return mensaje

interface = gr.Interface(fn=predict_rating, 
                     inputs=[gr.inputs.Textbox(line=1,placeholder="ingrese su número de usuario aquí"), 
                             gr.inputs.Textbox(line=1,placeholder="ingrese el id de la película")], 
                     outputs="text")
interface.launch()

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4cb4c04aa9040fab1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
